In [0]:
from delta.tables import *
stage_table_name='incremental_load.default.orders_stage'
target_table_name='incremental_load.default.orders_target'

In [0]:
stage_df= spark.read.table(stage_table_name)

In [0]:
if not spark.catalog.tableExists(target_table_name):
    stage_df.write.format('delta').saveAsTable(target_table_name)
else:
    target_table=DeltaTable.forName(spark, target_table_name)
    merge_condition= "stage.tracking_num = target.tracking_num"
    target_table.alias('target').merge(stage_df.alias('stage'), merge_condition).whenMatchedDelete().execute()
    stage_df.write.format('delta').mode('append').saveAsTable(target_table_name)